In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 10.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=94ccfa29a4ebe7281fb8a779fd22744d49c64a3ee65f970447c672ef5587b624
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
!pip install pyarrow
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [4]:
spark

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2019-02.parquet

--2023-03-18 06:10:40--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2019-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 65.8.245.178, 65.8.245.51, 65.8.245.171, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|65.8.245.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 513054623 (489M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2019-02.parquet’

fhvhv_tripdata_2019 100%[===================>] 489.29M  56.0MB/s    in 7.7s    

2023-03-18 06:10:48 (63.6 MB/s) - ‘fhvhv_tripdata_2019-02.parquet’ saved [513054623/513054623]



In [6]:
!wc -l fhvhv_tripdata_2019-02.parquet

1640819 fhvhv_tripdata_2019-02.parquet


In [7]:
import pandas as pd
df = pd.read_parquet('fhvhv_tripdata_2019-02.parquet')
df.to_csv('fhvhv_tripdata_2019-02.csv')

In [8]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2019-02.csv')

In [9]:
df.schema

StructType([StructField('_c0', StringType(), True), StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', StringType(), True), StructField('on_scene_datetime', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('trip_miles', StringType(), True), StructField('trip_time', StringType(), True), StructField('base_passenger_fare', StringType(), True), StructField('tolls', StringType(), True), StructField('bcf', StringType(), True), StructField('sales_tax', StringType(), True), StructField('congestion_surcharge', StringType(), True), StructField('airport_fee', StringType(), True), StructField('tips', StringType(), True), StructField('driver_pay', StringType(), True), 

In [10]:
!head -n 1001 fhvhv_tripdata_2019-02.csv > head.csv

In [11]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas.dtypes

Unnamed: 0                int64
hvfhs_license_num        object
dispatching_base_num     object
originating_base_num     object
request_datetime         object
on_scene_datetime        object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
trip_miles              float64
trip_time                 int64
base_passenger_fare     float64
tolls                   float64
bcf                     float64
sales_tax               float64
congestion_surcharge    float64
airport_fee             float64
tips                    float64
driver_pay              float64
shared_request_flag      object
shared_match_flag        object
access_a_ride_flag       object
wav_request_flag         object
wav_match_flag          float64
dtype: object

In [13]:
df_pandas.isnull().sum()

Unnamed: 0                 0
hvfhs_license_num          0
dispatching_base_num       0
originating_base_num     184
request_datetime          27
on_scene_datetime        271
pickup_datetime            0
dropoff_datetime           0
PULocationID               0
DOLocationID               0
trip_miles                 0
trip_time                  0
base_passenger_fare        0
tolls                      0
bcf                        0
sales_tax                  0
congestion_surcharge      87
airport_fee             1000
tips                       0
driver_pay                 0
shared_request_flag        0
shared_match_flag          0
access_a_ride_flag         0
wav_request_flag           0
wav_match_flag          1000
dtype: int64

In [14]:
df_pandas["originating_base_num"] = df_pandas["originating_base_num"].fillna("Nan values")
df_pandas["on_scene_datetime"] = df_pandas["on_scene_datetime"].fillna("Nan values")
df_pandas["request_datetime"] = df_pandas["request_datetime"].fillna("Nan values")
df_pandas["congestion_surcharge"] = df_pandas["congestion_surcharge"].dropna()
df_pandas["airport_fee"] = df_pandas["airport_fee"].dropna()
df_pandas["wav_match_flag"] = df_pandas["wav_match_flag"].dropna()
df_pandas.isnull().sum()

Unnamed: 0                 0
hvfhs_license_num          0
dispatching_base_num       0
originating_base_num       0
request_datetime           0
on_scene_datetime          0
pickup_datetime            0
dropoff_datetime           0
PULocationID               0
DOLocationID               0
trip_miles                 0
trip_time                  0
base_passenger_fare        0
tolls                      0
bcf                        0
sales_tax                  0
congestion_surcharge      87
airport_fee             1000
tips                       0
driver_pay                 0
shared_request_flag        0
shared_match_flag          0
access_a_ride_flag         0
wav_request_flag           0
wav_match_flag          1000
dtype: int64

In [15]:
from pyspark.sql import types

In [16]:
schema = types.StructType([
    types.StructField('', types.IntegerType(), True),
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('originating_base_num', types.StringType(), True),
    types.StructField('request_datetime', types.StringType(), True),
    types.StructField('on_scene_datetime', types.StringType(), True),
    types.StructField('pickup_datetime', types.StringType(), True),
    types.StructField('dropoff_datetime', types.StringType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('trip_miles', types.FloatType(), True),
    types.StructField('trip_time', types.IntegerType(), True),
    types.StructField('base_passenger_fare', types.FloatType(), True),
    types.StructField('tolls', types.FloatType(), True),
    types.StructField('bcf', types.FloatType(), True),
    types.StructField('sales_tax', types.FloatType(), True),
    types.StructField('congestion_surcharge', types.FloatType(), True),
    types.StructField('airport_fee', types.FloatType(), True),
    types.StructField('tips', types.FloatType(), True),
    types.StructField('driver_pay', types.FloatType(), True),
    types.StructField('shared_request_flag', types.StringType(), True),
    types.StructField('shared_match_flag', types.StringType(), True),
    types.StructField('access_a_ride_flag', types.StringType(), True),
    types.StructField('wav_request_flag', types.StringType(), True),
    types.StructField('wav_match_flag', types.StringType(), True)
])

In [17]:
spark.createDataFrame(df_pandas, schema=schema)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


DataFrame[: int, hvfhs_license_num: string, dispatching_base_num: string, originating_base_num: string, request_datetime: string, on_scene_datetime: string, pickup_datetime: string, dropoff_datetime: string, PULocationID: int, DOLocationID: int, trip_miles: float, trip_time: int, base_passenger_fare: float, tolls: float, bcf: float, sales_tax: float, congestion_surcharge: float, airport_fee: float, tips: float, driver_pay: float, shared_request_flag: string, shared_match_flag: string, access_a_ride_flag: string, wav_request_flag: string, wav_match_flag: string]

In [18]:
df1 = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2019-02.csv')

In [19]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: string (nullable = true)
 |-- on_scene_datetime: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- trip_miles: string (nullable = true)
 |-- trip_time: string (nullable = true)
 |-- base_passenger_fare: string (nullable = true)
 |-- tolls: string (nullable = true)
 |-- bcf: string (nullable = true)
 |-- sales_tax: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: string (nullable = true)
 |-- tips: string (nullable = true)
 |-- driver_pay: string (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- ac

In [20]:
df1.show()

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|   |hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+--

In [36]:
import pyspark
# make new column

# split() function defining parameters
split_cols = pyspark.sql.functions.split(df1['pickup_datetime'], ' ')
  
# Now applying split() using withColumn()
df2 = df1.withColumn('Date', split_cols.getItem(0))

In [49]:
df2.show()

+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+----------+
|   |hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|      Date|
+---+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+-

In [38]:
df2.count()

20159102

1. How many taxi trips were there on February 15?

In [48]:
df2.filter(df2.Date == "2019-02-15").count()

742396

2. Find the longest trip for each day ?

In [58]:
from pyspark.sql import functions as F

df2.groupBy("Date").agg(F.max("trip_miles")).show(28)

+----------+---------------+
|      Date|max(trip_miles)|
+----------+---------------+
|2019-02-01|         233.35|
|2019-02-02|         226.21|
|2019-02-04|         281.88|
|2019-02-03|         349.28|
|2019-02-05|          214.7|
|2019-02-06|         194.96|
|2019-02-07|         245.35|
|2019-02-08|         252.23|
|2019-02-09|         216.11|
|2019-02-10|         246.68|
|2019-02-11|         385.41|
|2019-02-12|         250.46|
|2019-02-13|         302.99|
|2019-02-14|          212.2|
|2019-02-15|         251.57|
|2019-02-16|         252.76|
|2019-02-17|         285.04|
|2019-02-18|         469.26|
|2019-02-19|         300.64|
|2019-02-20|         273.62|
|2019-02-21|         227.55|
|2019-02-22|         261.03|
|2019-02-23|         270.04|
|2019-02-24|         460.84|
|2019-02-25|         397.62|
|2019-02-26|         250.81|
|2019-02-27|         264.34|
|2019-02-28|         224.49|
+----------+---------------+



3. Find Top 5 Most frequent `dispatching_base_num` ?

In [63]:
from pyspark.sql.functions import col, desc

df2.groupBy("dispatching_base_num").count().sort(col("count").desc()).show(5)

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|4690874|
|              B02764|1559308|
|              B02914| 979266|
|              B02765| 978081|
|              B02800| 975850|
+--------------------+-------+
only showing top 5 rows



4. Find Top 5 Most common location pairs (PUlocationID and DOlocationID) ?

In [121]:
df_PU_DO = df2.select("PUlocationID", "DOlocationID").where("PUlocationID == DOlocationID")
df_PU_DO.groupBy("PUlocationID", "DOlocationID").count().sort(col("count").desc()).show(5)


+------------+------------+-----+
|PUlocationID|DOlocationID|count|
+------------+------------+-----+
|          76|          76|43371|
|          26|          26|37983|
|          61|          61|33979|
|          39|          39|31662|
|           7|           7|25950|
+------------+------------+-----+
only showing top 5 rows

